In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
from io import StringIO


base_url = "https://www.thegradcafe.com/survey/?per_page=40&q=&institution={school}&program={program}&degree=Masters&page={page}"

In [ ]:
schools = [
    'Harvard University',
    'Columbia University',
    'Yale University',
    'University of Pennsylvania',
    'Princeton University',
    'Cornell University',
    'University of Michigan',
    'University of Texas',
    'Stanford University',
    'University+of+California-Los+Angeles',
    'University+of+Miami',
]

programs = [
    'Applied Analytics',
    'Mathematics',
    'Computer Science',
    'Statistics',
    'History',
    'English',
    'Journalism',
    'Social Work',
    'Software Engineering',
    'Data Science',
    'Political Science',
    'Law',
    'Medicine'
]

In [ ]:

all_data = pd.DataFrame()
for school in schools:
    for program in programs:
        page=1
        #Init Page
        url = base_url.format(
            school=school,
            program=program,
            page=page
        )
        headers = {
            'User-agent':'my-app'
        }
        response = requests.get(
            url,
            headers=headers
            )
        soup = BeautifulSoup(response.text)
        pages = max([int(i.text) for i in soup.findAll('nav')[-1].findAll('a') if 'Next' not in i.text])

        for page in range(1,pages+1):

            url = base_url.format(
                school=school,
                program=program,
                page=page
            )
            headers = {
                'User-agent':'my-app'
            }
            response = requests.get(
                url,
                headers=headers
                )
            soup = BeautifulSoup(response.text)

            tbls = soup.findAll('table')
            tblhtml = tbls[0]

            rows = tblhtml.findAll('tr',attrs={'class':''})

            thead = tblhtml.findAll('thead')

            rowstr = ''.join([i.__str__() for i in rows])

            tbl_string = '<table>'+thead[0].__str__()+"<tbody>{rowstr}</tbody>".format(rowstr=rowstr)+"</table>"

            tbl = pd.read_html(StringIO(tbl_string.\
                        replace("%","")))[0]#tw-border-none

            result = tbl[~(tbl['School']=='School')&~(tbl['School'].str.contains('document'))][
                ['School','Program','Added On','Decision']
            ].copy()

            print(school,program,page)

            result['School Search'] = school.replace("+"," ")
            result['Program Search'] = program.replace("+"," ")
            all_data = pd.concat([all_data,result])

In [ ]:


school = 'Columbia University'.replace(" ","+")
program = 'Computer Science'.replace(" ","+")
page=1

In [61]:
#Init Page
url = base_url.format(
    school=school,
    program=program,
    page=page
)
headers = {
    'User-agent':'my-app'
}
response = requests.get(
    url,
    headers=headers
    )
soup = BeautifulSoup(response.text)
pages = max([int(i.text) for i in soup.findAll('nav')[-1].findAll('a') if 'Next' not in i.text])

In [ ]:
all_data = pd.DataFrame()

for page in range(1,pages+1):

    url = base_url.format(
        school=school,
        program=program,
        page=page
    )
    headers = {
        'User-agent':'my-app'
    }
    response = requests.get(
        url,
        headers=headers
        )
    soup = BeautifulSoup(response.text)

    tbls = soup.findAll('table')
    tblhtml = tbls[0]

    rows = tblhtml.findAll('tr',attrs={'class':''})

    thead = tblhtml.findAll('thead')

    rowstr = ''.join([i.__str__() for i in rows])

    tbl_string = '<table>'+thead[0].__str__()+"<tbody>{rowstr}</tbody>".format(rowstr=rowstr)+"</table>"

    tbl = pd.read_html(StringIO(tbl_string.\
                replace("%","")))[0]#tw-border-none

    result = tbl[~(tbl['School']=='School')&~(tbl['School'].str.contains('document'))][
        ['School','Program','Added On','Decision']
    ].copy()

    all_data = pd.concat([all_data,result])
    print(page)

all_data['School Search'] = school.replace("+"," ")
all_data['Program Search'] = program.replace("+"," ")

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22


In [63]:
all_data

,School,Program,Added On,Decision
1,Columbia University,Computer Science Masters,"April 26, 2024",Rejected on 26 Apr
2,Columbia University,Computer Science Masters,"April 26, 2024",Rejected on 26 Apr
3,Columbia University,Computer Science Masters,"April 26, 2024",Rejected on 26 Apr
4,Columbia University,Computer Science Masters,"April 23, 2024",Rejected on 23 Apr
5,Columbia University,Computer Science Masters,"April 20, 2024",Rejected on 19 Apr
...,...,...,...,...
19,Columbia University (Fu Foundation School Of E...,Computer Science Masters,"November 30, -0001",Rejected on 30 Nov
20,Columbia University (GSAS),Computer Science Masters,"November 30, -0001",Interview on 30 Nov
21,Columbia University (Fu Foundation School Of E...,Computer Science Masters,"November 30, -0001",Rejected on 30 Nov
22,Columbia University (Fu Foundation School Of E...,Computer Science - Data Science Masters,"November 30, -0001",Other on 30 Nov
